In [12]:
import pandas as pd

import sys

sys.path.append("./..")
from src.utils.get_youtube_url_official import find_urls_for_songs

In [15]:
df = pd.read_csv("../data/top_songs_curated.csv")

In [16]:
df[df["youtube_title"].isna()].head(3)

,Track URI,Track Name,Artist URI(s),Artist Name(s),Album URI,Album Name,Album Artist URI(s),Album Artist Name(s),Album Release Date,Album Image URL,...,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Album Genres,Label,Copyrights,youtube_title,youtube_url
0,spotify:track:7iL6o9tox1zgHpKUfh9vuC,In Da Club,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:5G5rgQHzdQnw32SI0WjIo5,Get Rich Or Die Tryin',spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2003-02-06,https://i.scdn.co/image/ab67616d0000b273d843fa...,...,0.00000,0.0708,0.777,90.051,4.0,NaN,Shady Records,C © 2003 Shady Records/Aftermath Records/Inter...,NaN,NaN
1,spotify:track:7xp7FkbxGEWcVDvL1KayoD,In Da Club - Single Version / Edit,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:67ma4XvpbAfD4UpUPRwRN4,RnB Fridays Vol. 3,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,Various Artists,2017-04-21,https://i.scdn.co/image/ab67616d0000b273234776...,...,0.00000,0.0951,0.702,89.999,4.0,NaN,Universal Music Australia Pty. Ltd.,"C © 2017 Universal Music Australia Pty Ltd., P...",NaN,NaN
2,spotify:track:0x7U32vZzq7e1qVpA0MBwK,Just A Lil Bit,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:3PdtIrHMhNPNGV2NQAlTuU,The Massacre,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2005-03-03,https://i.scdn.co/image/ab67616d0000b273edc803...,...,0.00608,0.3150,0.527,96.946,4.0,NaN,Universal Music Group,C © 2004 Shady Records/Aftermath Records/Inter...,NaN,NaN


In [46]:
def get_uri_for_curated(number_of_songs=4):
    df = pd.read_csv("../data/top_songs_curated.csv")
    df_missing = df[df["youtube_title"].isna()]

    df_missing_n = df_missing.head(number_of_songs)

    songs = (
        df_missing_n[["Track Name", "Artist Name(s)"]]
        .rename(columns={"Track Name": "title", "Artist Name(s)": "artist"})
        .to_dict(orient="records")
    )

    results = find_urls_for_songs(songs)
    results = pd.DataFrame.from_dict(results, orient="index")

    merged_df = df.merge(
        results,
        how="left",
        left_on=["Track Name", "Artist Name(s)"],
        right_on=["title", "artist"],
        suffixes=("", "_drop"),
    )

    merged_df["youtube_title"] = merged_df["youtube_title"].combine_first(
        merged_df["youtube_title_drop"]
    )
    merged_df["youtube_url"] = merged_df["youtube_url"].combine_first(
        merged_df["youtube_url_drop"]
    )
    merged_df = merged_df.drop(
        columns=["title", "artist", "youtube_title_drop", "youtube_url_drop"]
    )

    merged_df = merged_df.sort_values(["Artist Name(s)", "Track Name"])
    merged_df.to_csv("../data/top_songs_curated.csv", index=False)

In [18]:
df = pd.read_csv("../data/top_songs_curated.csv")
df_missing = df[df["youtube_title"].isna()]

df_missing_n = df_missing.head(3)

songs = (
    df_missing_n[["Track Name", "Artist Name(s)"]]
    .rename(columns={"Track Name": "title", "Artist Name(s)": "artist"})
    .to_dict(orient="records")
)

results = find_urls_for_songs(songs)
results = pd.DataFrame.from_dict(results, orient="index")
results

API key loaded successfully!
Searching for 'In Da Club' by 50 Cent...
Searching for 'In Da Club - Single Version / Edit' by 50 Cent...
Searching for 'Just A Lil Bit' by 50 Cent...


,title,artist,youtube_title,youtube_url
In Da Club - 50 Cent,In Da Club,50 Cent,50 Cent - In Da Club (Official Music Video),https://www.youtube.com/watch?v=5qm8PH4xAss
In Da Club - Single Version / Edit - 50 Cent,In Da Club - Single Version / Edit,50 Cent,50 Cent - In Da Club (Lyrics),https://www.youtube.com/watch?v=H3axP4norfk
Just A Lil Bit - 50 Cent,Just A Lil Bit,50 Cent,50 Cent - Just A Lil Bit,https://www.youtube.com/watch?v=GllEDACUbNo


In [44]:
merged_df = df.merge(
    results,
    how="left",
    left_on=["Track Name", "Artist Name(s)"],
    right_on=["title", "artist"],
    suffixes=("", "_drop"),
).drop(columns=["title", "artist"])
merged_df

,Track URI,Track Name,Artist URI(s),Artist Name(s),Album URI,Album Name,Album Artist URI(s),Album Artist Name(s),Album Release Date,Album Image URL,...,Valence,Tempo,Time Signature,Album Genres,Label,Copyrights,youtube_title,youtube_url,youtube_title_drop,youtube_url_drop
0,spotify:track:7iL6o9tox1zgHpKUfh9vuC,In Da Club,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:5G5rgQHzdQnw32SI0WjIo5,Get Rich Or Die Tryin',spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2003-02-06,https://i.scdn.co/image/ab67616d0000b273d843fa...,...,0.777,90.051,4.0,NaN,Shady Records,C © 2003 Shady Records/Aftermath Records/Inter...,NaN,NaN,50 Cent - In Da Club (Official Music Video),https://www.youtube.com/watch?v=5qm8PH4xAss
1,spotify:track:7xp7FkbxGEWcVDvL1KayoD,In Da Club - Single Version / Edit,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:67ma4XvpbAfD4UpUPRwRN4,RnB Fridays Vol. 3,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,Various Artists,2017-04-21,https://i.scdn.co/image/ab67616d0000b273234776...,...,0.702,89.999,4.0,NaN,Universal Music Australia Pty. Ltd.,"C © 2017 Universal Music Australia Pty Ltd., P...",NaN,NaN,50 Cent - In Da Club (Lyrics),https://www.youtube.com/watch?v=H3axP4norfk
2,spotify:track:0x7U32vZzq7e1qVpA0MBwK,Just A Lil Bit,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:3PdtIrHMhNPNGV2NQAlTuU,The Massacre,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2005-03-03,https://i.scdn.co/image/ab67616d0000b273edc803...,...,0.527,96.946,4.0,NaN,Universal Music Group,C © 2004 Shady Records/Aftermath Records/Inter...,NaN,NaN,50 Cent - Just A Lil Bit,https://www.youtube.com/watch?v=GllEDACUbNo
3,spotify:track:3G0yz3DZn3lfraledmBCT0,P.I.M.P.,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:5G5rgQHzdQnw32SI0WjIo5,Get Rich Or Die Tryin',spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2003-02-06,https://i.scdn.co/image/ab67616d0000b273d843fa...,...,0.848,84.722,4.0,NaN,Shady Records,C © 2003 Shady Records/Aftermath Records/Inter...,NaN,NaN,NaN,NaN
4,spotify:track:1N9JazWqQEGOtcDfL0IAaK,Window Shopper,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:1ExH71fC9bKxWPM9wFzoTK,Best Of 50 Cent,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2017-03-31,https://i.scdn.co/image/ab67616d0000b273f46bc3...,...,0.484,174.199,4.0,NaN,Universal Music Group,C © 2017 Shady Records/Aftermath Records/Inter...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,spotify:track:7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:5qt11cWjSs5Gbqj2Wyfu38,Enema Of The State,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,1999-01-01,https://i.scdn.co/image/ab67616d0000b273645606...,...,0.695,148.599,4.0,NaN,Interscope,"C © 1999 Geffen Records, P ℗ 1999 Geffen Records",NaN,NaN,NaN,NaN
1102,spotify:track:2TWTOOE397zriKCKwGLYgI,Bored To Death,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:2GF8wZ1o0NpHSFhjgz6eBD,California,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,2016-07-01,https://i.scdn.co/image/ab67616d0000b273cf72e7...,...,0.638,159.948,4.0,NaN,Liberator Music,C 2016 Viking Wizard Eyes LLC under exclusive ...,NaN,NaN,NaN,NaN
1103,spotify:track:1tT1tIZAINzFZyjoPKtg7C,Dammit,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:5Rb6Q94DDEPWakydob5bPp,Can't Hardly Wait,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,Various Artists,1998-05-26,https://i.scdn.co/image/ab67616d0000b273603b20...,...,0.287,110.487,4.0,NaN,Elektra Records,"C © 1998 Columbia Pictures Industries, Inc., P...",NaN,NaN,NaN,NaN
1104,spotify:track:5dhQCqONiQji7k4RkhIcjq,I Miss You,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:2HoSi9LK61wIgBUJS8v6XX,blink-182 (explicit version) [Explicit Version],spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,2003-11-18,https://i.scdn.co/image/ab67616d0000b27376f761...,...,0.611,109.997,4.0,NaN,Geffen,"C (C) 2003 Geffen Records, P (P) 2003 Geffen R...",NaN,NaN,NaN,NaN


In [45]:
merged_df["youtube_title"] = merged_df["youtube_title"].combine_first(
    merged_df["youtube_title_drop"]
)
merged_df["youtube_url"] = merged_df["youtube_url"].combine_first(
    merged_df["youtube_url_drop"]
)
merged_df = merged_df.drop(columns=["youtube_title_drop"])
merged_df = merged_df.drop(columns=["youtube_url_drop"])
merged_df

,Track URI,Track Name,Artist URI(s),Artist Name(s),Album URI,Album Name,Album Artist URI(s),Album Artist Name(s),Album Release Date,Album Image URL,...,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Album Genres,Label,Copyrights,youtube_title,youtube_url
0,spotify:track:7iL6o9tox1zgHpKUfh9vuC,In Da Club,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:5G5rgQHzdQnw32SI0WjIo5,Get Rich Or Die Tryin',spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2003-02-06,https://i.scdn.co/image/ab67616d0000b273d843fa...,...,0.000000,0.0708,0.777,90.051,4.0,NaN,Shady Records,C © 2003 Shady Records/Aftermath Records/Inter...,50 Cent - In Da Club (Official Music Video),https://www.youtube.com/watch?v=5qm8PH4xAss
1,spotify:track:7xp7FkbxGEWcVDvL1KayoD,In Da Club - Single Version / Edit,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:67ma4XvpbAfD4UpUPRwRN4,RnB Fridays Vol. 3,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,Various Artists,2017-04-21,https://i.scdn.co/image/ab67616d0000b273234776...,...,0.000000,0.0951,0.702,89.999,4.0,NaN,Universal Music Australia Pty. Ltd.,"C © 2017 Universal Music Australia Pty Ltd., P...",50 Cent - In Da Club (Lyrics),https://www.youtube.com/watch?v=H3axP4norfk
2,spotify:track:0x7U32vZzq7e1qVpA0MBwK,Just A Lil Bit,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:3PdtIrHMhNPNGV2NQAlTuU,The Massacre,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2005-03-03,https://i.scdn.co/image/ab67616d0000b273edc803...,...,0.006080,0.3150,0.527,96.946,4.0,NaN,Universal Music Group,C © 2004 Shady Records/Aftermath Records/Inter...,50 Cent - Just A Lil Bit,https://www.youtube.com/watch?v=GllEDACUbNo
3,spotify:track:3G0yz3DZn3lfraledmBCT0,P.I.M.P.,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:5G5rgQHzdQnw32SI0WjIo5,Get Rich Or Die Tryin',spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2003-02-06,https://i.scdn.co/image/ab67616d0000b273d843fa...,...,0.000004,0.0368,0.848,84.722,4.0,NaN,Shady Records,C © 2003 Shady Records/Aftermath Records/Inter...,NaN,NaN
4,spotify:track:1N9JazWqQEGOtcDfL0IAaK,Window Shopper,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,spotify:album:1ExH71fC9bKxWPM9wFzoTK,Best Of 50 Cent,spotify:artist:3q7HBObVc0L8jNeTe5Gofh,50 Cent,2017-03-31,https://i.scdn.co/image/ab67616d0000b273f46bc3...,...,0.000000,0.4890,0.484,174.199,4.0,NaN,Universal Music Group,C © 2017 Shady Records/Aftermath Records/Inter...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,spotify:track:7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:5qt11cWjSs5Gbqj2Wyfu38,Enema Of The State,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,1999-01-01,https://i.scdn.co/image/ab67616d0000b273645606...,...,0.000000,0.5470,0.695,148.599,4.0,NaN,Interscope,"C © 1999 Geffen Records, P ℗ 1999 Geffen Records",NaN,NaN
1102,spotify:track:2TWTOOE397zriKCKwGLYgI,Bored To Death,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:2GF8wZ1o0NpHSFhjgz6eBD,California,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,2016-07-01,https://i.scdn.co/image/ab67616d0000b273cf72e7...,...,0.000002,0.1170,0.638,159.948,4.0,NaN,Liberator Music,C 2016 Viking Wizard Eyes LLC under exclusive ...,NaN,NaN
1103,spotify:track:1tT1tIZAINzFZyjoPKtg7C,Dammit,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:5Rb6Q94DDEPWakydob5bPp,Can't Hardly Wait,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,Various Artists,1998-05-26,https://i.scdn.co/image/ab67616d0000b273603b20...,...,0.000167,0.1000,0.287,110.487,4.0,NaN,Elektra Records,"C © 1998 Columbia Pictures Industries, Inc., P...",NaN,NaN
1104,spotify:track:5dhQCqONiQji7k4RkhIcjq,I Miss You,spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,spotify:album:2HoSi9LK61wIgBUJS8v6XX,blink-182 (explicit version) [Explicit Version],spotify:artist:6FBDaR13swtiWwGhX1WQsP,blink-182,2003-11-18,https://i.scdn.co/image/ab67616d0000b27376f761...,...,0.000007,0.0835,0.611,109.997,4.0,NaN,Geffen,"C (C) 20

In [34]:
merged_df = merged_df.sort_values(["Artist Name(s)", "Track Name"]).drop_duplicates(
    ["Artist Name(s)", "Track Name"]
)

merged_df.to_csv("../data/top_songs_curated.csv", index=False)

In [47]:
get_uri_for_curated(number_of_songs=2)

API key loaded successfully!
Searching for 'P.I.M.P.' by 50 Cent...
Searching for 'Window Shopper' by 50 Cent...
